In [2]:
import numpy as np
import pandas as pd
import yfinance as yf # Yahoo Finance API
from datetime import datetime
from pprint import pprint # make printing data more readable
from treasury.client import FederalTreasuryClient # Fedearal Open Source Treasury API
import plotly.graph_objs as graph # just in case we need to graph
# ticker_symbol_input = yf.Ticker("MSFT")
# print(ticker_symbol_input.info)

In [3]:
# parent/super class
class EDWARD_dashboard:
    def __init__(self, ticker_symbol: str,  contract_type: str, exercise_price: int, expiration_date: str):
        self.ticker_symbol = ticker_symbol.upper()
        self.current_date = datetime.now().strftime("%Y-%m-%d")
        self.contract_type = contract_type
        self.exercise_price = exercise_price
        self.expiration_date = expiration_date

    def days_between(self):
        d1, d2 = datetime.strptime(self.current_date, "%Y-%m-%d"), datetime.strptime(self.expiration_date, "%Y-%m-%d")
        diff_times = abs((d2 - d1).days)
        return diff_times

    def market_price(self):
        ticker_symbol = yf.Ticker(self.ticker_symbol).info
        market_price = ticker_symbol['regularMarketPrice']
        return market_price # spot price

    def risk_free_rate(self):
        # still need to use API to find nominal_rate and inflation_rate
        risk_free_rate = ((1+nominal_rate)/(1+inflation_rate))
        return risk_free_rate

    def dividend_yield(self):
        ticker_symbol = yf.Ticker(self.ticker_symbol).info
        dividend_rate = ticker_symbol['dividendRate']
        if dividend_rate == None:
            dividend_rate = 0
        return dividend_rate

    def options_data(self, type): # type can by calls or puts
        try:
            option_data = yf.Ticker(self.ticker_symbol).option_chain(date=self.expiration_date)
            print(f"Could find a matching expiration date for {self.ticker_symbol} {type} options")
        except ValueError:
            print(f"Expirations date chosen isn't available for this stock: {self.ticker_symbol} Here is list of Available {type} options ")
            option_data = yf.Ticker(self.ticker_symbol).option_chain()
        if type == 'calls':
            return option_data.calls
        elif type =='puts':
            return option_data.puts

    def futures_formula(self):
        Price_contract =  self.market_price()*np.e**((self.risk_free_rate()-self.dividend_yield())*(self.days_between()))
        return Price_contract

    def relationship_futuresprice_exepctedspotprice(self, k): #k is an investor’s required return for this investment. Ask  if formula is right
        # ask if this method is neccessary or repetitive of the futures_method and also incomplete method at the moment
        indicator = np.e**((self.risk_free_rate()-k) * self.days_between())
        if indicator == 0:
            return f"No systematic risk"
        elif indicator < 0:
            return f"Negative sytematic risk"
        elif indicator > 0:
            return f"Positive systematic risk"
        pass

    def portfolio(self):
        ticker_symbol = yf.Ticker(self.ticker_symbol)
        pass

    def __str__(self):
        return f"Current date {str(self.current_date)}; difference between dates {str(self.days_between())}; upper case ticker {self.ticker_symbol}; market {self.market_price()}; dividend rate {self.dividend_yield()}: "

tesla = EDWARD_dashboard("tSlA","future",300,"2022-11-11")
print(tesla)
# print(tesla.futures_formula())
# tesla.options_data('calls')
tesla.options_data('puts')


Current date 2022-11-07; difference between dates 4; upper case ticker TSLA; market 202.8339; dividend rate 0: 
Could find a matching expiration date for TSLA puts options


,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,TSLA221111P00100000,2022-11-07 14:44:40+00:00,100.0,0.01,0.00,0.01,-0.01,-50.000000,26.0,10574,1.875001,False,REGULAR,USD
1,TSLA221111P00110000,2022-11-07 14:31:26+00:00,110.0,0.01,0.01,0.02,-0.02,-66.666670,681.0,1052,1.781251,False,REGULAR,USD
2,TSLA221111P00120000,2022-11-07 14:41:01+00:00,120.0,0.02,0.02,0.04,0.00,0.000000,30.0,2185,1.648439,False,REGULAR,USD
3,TSLA221111P00130000,2022-11-07 14:39:56+00:00,130.0,0.04,0.03,0.04,-0.01,-20.000002,1030.0,3841,1.437503,False,REGULAR,USD
4,TSLA221111P00140000,2022-11-07 14:48:22+00:00,140.0,0.05,0.04,0.05,-0.01,-16.666664,1403.0,3693,1.250004,False,REGULAR,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,TSLA221111P00335000,2022-11-02 18:00:21+00:00,335.0,112.20,130.90,131.75,0.00,0.000000,1.0,0,0.000010,True,REGULAR,USD
58,TSLA221111P00340000,2022-11-03 14:22:38+00:00,340.0,122.30,138.20,138.85,0.00,0.000000,2.0,0,2.553715,True,REGULAR,USD
59,TSLA221111P00345000,2022-11-03 15:04:10+00:00,345.0,124.50,142.85,143.65,0.00,0.000000,NaN,0,2.499027,True,REGULAR,USD
60,TSLA221111P00350000,2022-10-20 19:21:41+00:00,350.0,142.95,146.50,147.50,0.00,0.000000,31.0,0,2.099614,True,REGULAR,USD


In [ ]:
# User Inputs
input_symbol = input("Ticker Symbol you would like to use? ")
# ticker_symbol = yf.Ticker(input_symbol)
input_contract_type = input("What contracct type would you like to use (Futures, Options, Swaps)? ")
input_exercise_price = int(input("What price would you like to exercise the shares at? "))
input_expiration_date = input("What date would you like to set the expiration? (Y-M-D)")
# EDWARD_dashboard(input_symbol, input_contract_type, input_exercise_price, input_expiration_date)


In [5]:
"""
Risk Free Rate Calculator based on 3-Month US treasury Bill Rates
Author: ranaroussi
link: https://gist.github.com/ranaroussi/72d0e92bbe31d1514baccf00175049e4
"""
# de-annualize yearly interest rates
def deannualize(annual_rate, periods=365):
    return (1 + annual_rate) ** (1/periods) - 1

def get_risk_free_rate():
    # download 3-month us treasury bills rates
    annualized = yf.download("^IRX")["Adj Close"]

    # de-annualize
    daily = annualized.apply(deannualize)

    # create dataframe
    return pd.DataFrame({"annualized": annualized, "daily": daily})

if __name__ == "__main__":
    rates = get_risk_free_rate()
    print(rates.tail())

[*********************100%***********************]  1 of 1 completed
            annualized     daily
Date                            
2022-11-01       4.050  0.004447
2022-11-02       4.025  0.004433
2022-11-03       4.025  0.004433
2022-11-04       4.013  0.004426
2022-11-07       4.000  0.004419
